In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def loss_fn_kd(outputs, labels, teacher_outputs, params):

    alpha = params.alpha
    T = params.temperature

    # 1. 증류 손실 (Distillation Loss): 교사 모델의 지식을 모방
    # - 학생 모델의 출력에는 log_softmax를 적용 (KLDivLoss 요구사항)
    # - 교사 모델의 출력에는 softmax를 적용
    # - T(온도)로 나누어 분포를 부드럽게(soft) 만듦
    # - T^2를 곱하는 이유는 그라디언트 스케일을 맞추기 위함 (Hinton et al. 논문 참조)
    distillation_loss = nn.KLDivLoss(reduction='batchmean')(
        F.log_softmax(outputs/T, dim=1),
        F.softmax(teacher_outputs/T, dim=1)
    ) * (alpha * T * T)

    # 2. 학생 손실 (Student Loss): 정답 레이블(Hard label)과의 차이
    # - 일반적인 Cross Entropy Loss 사용
    student_loss = F.cross_entropy(outputs, labels) * (1. - alpha)

    # 최종 손실 합산
    KD_loss = distillation_loss + student_loss

    return KD_loss

In [ ]:
def train_kd(model, teacher_model, optimizer, loss_fn_kd, dataloader, metrics, params):
    """
    `num_steps` 배치만큼 모델을 학습시킵니다.

    Args:
        model: (torch.nn.Module) 학습할 신경망 (학생 모델)
        optimizer: (torch.optim) 모델 파라미터 최적화 도구
        loss_fn_kd: 지식 증류 손실 함수
        dataloader: 데이터 로더
        metrics: (dict) 평가 지표 딕셔너리
        params: (Params) 하이퍼파라미터 객체
    """

    # 모델을 학습 모드로 설정 (Dropout, BatchNorm 등이 학습 모드로 동작)
    model.train()
    # 교사 모델은 평가 모드로 설정 (가중치 업데이트 안 함, Dropout 비활성화)
    teacher_model.eval()

    # 현재 학습 루프의 요약 정보와 손실(loss) 이동 평균을 저장할 객체
    summ = []
    loss_avg = utils.RunningAverage()

    # 진행률 표시줄(tqdm) 사용
    with tqdm(total=len(dataloader)) as t:
        for i, (train_batch, labels_batch) in enumerate(dataloader):
            # GPU 사용 가능 시 데이터 이동
            if params.cuda:
                train_batch, labels_batch = train_batch.cuda(async=True), \
                                            labels_batch.cuda(async=True)
            # torch Variable로 변환 (참고: 최신 PyTorch에서는 불필요하지만 레거시 코드 호환용)
            train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)

            # 모델(학생) 출력 계산
            output_batch = model(train_batch)

            # 교사 모델의 출력 계산 (그라디언트 계산 불필요)
            with torch.no_grad():
                output_teacher_batch = teacher_model(train_batch)

            if params.cuda:
                output_teacher_batch = output_teacher_batch.cuda(async=True)

            # KD 손실 함수 계산 (학생 출력, 정답, 교사 출력 이용)
            loss = loss_fn_kd(output_batch, labels_batch, output_teacher_batch, params)

            # 이전 그라디언트 초기화 후, 손실에 대한 모든 변수의 그라디언트 계산 (역전파)
            optimizer.zero_grad()
            loss.backward()

            # 계산된 그라디언트를 사용하여 파라미터 업데이트
            optimizer.step()

            # 매번 평가하면 느리므로, 일정 주기마다(save_summary_steps) 요약 정보 평가
            if i % params.save_summary_steps == 0:
                # 데이터를 CPU로 옮기고 numpy 배열로 변환 (메트릭 계산용)
                output_batch = output_batch.data.cpu().numpy()
                labels_batch = labels_batch.data.cpu().numpy()

                # 현재 배치에 대해 모든 메트릭 계산
                summary_batch = {metric:metrics[metric](output_batch, labels_batch)
                                 for metric in metrics}
                summary_batch['loss'] = loss.data[0] # 주의: 최신 PyTorch는 loss.item() 권장
                summ.append(summary_batch)

            # 평균 손실 업데이트
            loss_avg.update(loss.data[0])

            # 진행률 표시줄에 현재 평균 손실 표시
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
            t.update()

    # 모든 배치의 메트릭 평균 계산
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    logging.info("- Train metrics: " + metrics_string)